In [159]:
import pyspark, re, json, os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col, regexp_replace, lower, explode, array_contains, collect_set, lit, coalesce, array, count, when, isnull, mean, rand
from pyspark.sql.types import IntegerType, StringType, FloatType, StructType, StructField
from pyspark.ml.feature import StringIndexer
import pandas as pd

In [160]:
# Initialize Spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

24/10/04 16:10:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [161]:
# Load the data from the JSON file
with open('../data/landing/domain_data.json', 'r') as f:
    data = json.load(f)

# Convert the JSON to a pandas DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

df

,name,cost_text,rooms,parking,description,prop_type,additional_features
https://www.domain.com.au/1001-39-queen-street-melbourne-vic-3000-17221118,"1001/39 Queen Street, Melbourne VIC 3000",$825 per week,"[3 Beds, 1 Bath]",[− Parking],This stunning residence in the historic Madiso...,Apartment / Unit / Flat,NaN
https://www.domain.com.au/2817s-220-spencer-street-melbourne-vic-3000-17223080,"2817s/220 Spencer Street, Melbourne VIC 3000",$800 per week,"[2 Beds, 2 Baths]",[1 Parking],"Stunning Furnished Two Bedroom, Two Bathroom a...",Apartment / Unit / Flat,[Furnished]
https://www.domain.com.au/3207-80-a-beckett-street-melbourne-vic-3000-17119770,"3207/80 A'Beckett Street, Melbourne VIC 3000",$760/w FURNISHED,"[2 Beds, 1 Bath]",[− Parking],The MY80 development gives you the best that t...,Apartment / Unit / Flat,"[Gym, Intercom, Built in wardrobes, Dishwasher..."
https://www.domain.com.au/1413-199-william-street-melbourne-vic-3000-17225646,"1413/199 William Street, Melbourne VIC 3000",$650,"[2 Beds, 1 Bath]",[− Parking],Excellent FULLY FURNISHED property not to miss...,Apartment / Unit / Flat,"[Balcony, Outdoor Entertainment Area, Swimming..."
https://www.domain.com.au/2505-371-little-lonsdale-street-melbourne-vic-3000-17224306,"2505/371 Little Lonsdale Street, Melbourne VIC...",$550,"[1 Bed, 1 Bath]",[− Parking],"380 Melbourne elevates style, convenience, and...",Apartment / Unit / Flat,NaN
...,...,...,...,...,...,...,...
https://www.domain.com.au/19-grazing-road-weir-views-vic-3338-17214974,"19 Grazing Road, Weir Views VIC 3338",$460 per week,"[3 Beds, 2 Baths]",[2 Parking],Spacious 3-bedroom open plan living home offer...,House,[Ensuite]
https://www.domain.com.au/9-norwood-avenue-weir-views-vic-3338-17111843,"9 Norwood Avenue, Weir Views VIC 3338",$460/week,"[4 Beds, 2 Baths]",[2 Parking],A perfect blend of comfort and modern living. ...,House,[]
https://www.domain.com.au/16-shackell-st-weir-views-vic-3338-17222155,"16 Shackell St, Weir Views VIC 3338",$460 weekly,"[4 Beds, 2 Baths]",[2 Parking],"Welcome to your new home at 16 Shackell St, We...",House,[]
https://www.domain.com.au/34-metroon-drive-weir-views-vic-3338-17204576,"34 Metroon Drive, Weir Views VIC 3338",$450,"[3 Beds, 2 Baths]",[2 Parking],The Roof Real Estate is proudly showcasing thi...,House,NaN


In [162]:
df_reset = df.reset_index().rename(columns={'index': 'property_url'})
df_reset

,property_url,name,cost_text,rooms,parking,description,prop_type,additional_features
0,https://www.domain.com.au/1001-39-queen-street...,"1001/39 Queen Street, Melbourne VIC 3000",$825 per week,"[3 Beds, 1 Bath]",[− Parking],This stunning residence in the historic Madiso...,Apartment / Unit / Flat,NaN
1,https://www.domain.com.au/2817s-220-spencer-st...,"2817s/220 Spencer Street, Melbourne VIC 3000",$800 per week,"[2 Beds, 2 Baths]",[1 Parking],"Stunning Furnished Two Bedroom, Two Bathroom a...",Apartment / Unit / Flat,[Furnished]
2,https://www.domain.com.au/3207-80-a-beckett-st...,"3207/80 A'Beckett Street, Melbourne VIC 3000",$760/w FURNISHED,"[2 Beds, 1 Bath]",[− Parking],The MY80 development gives you the best that t...,Apartment / Unit / Flat,"[Gym, Intercom, Built in wardrobes, Dishwasher..."
3,https://www.domain.com.au/1413-199-william-str...,"1413/199 William Street, Melbourne VIC 3000",$650,"[2 Beds, 1 Bath]",[− Parking],Excellent FULLY FURNISHED property not to miss...,Apartment / Unit / Flat,"[Balcony, Outdoor Entertainment Area, Swimming..."
4,https://www.domain.com.au/2505-371-little-lons...,"2505/371 Little Lonsdale Street, Melbourne VIC...",$550,"[1 Bed, 1 Bath]",[− Parking],"380 Melbourne elevates style, convenience, and...",Apartment / Unit / Flat,NaN
...,...,...,...,...,...,...,...,...
11753,https://www.domain.com.au/19-grazing-road-weir...,"19 Grazing Road, Weir Views VIC 3338",$460 per week,"[3 Beds, 2 Baths]",[2 Parking],Spacious 3-bedroom open plan living home offer...,House,[Ensuite]
11754,https://www.domain.com.au/9-norwood-avenue-wei...,"9 Norwood Avenue, Weir Views VIC 3338",$460/week,"[4 Beds, 2 Baths]",[2 Parking],A perfect blend of comfort and modern living. ...,House,[]
11755,https://www.domain.com.au/16-shackell-st-weir-...,"16 Shackell St, Weir Views VIC 3338",$460 weekly,"[4 Beds, 2 Baths]",[2 Parking],"Welcome to your new home at 16 Shackell St, We...",House,[]
11756,https://www.domain.com.au/34-metroon-drive-wei...,"34 Metroon Drive, Weir Views VIC 3338",$450,"[3 Beds, 2 Baths]",[2 Parking],The Roof Real Estate is proudly showcasing thi...,House,NaN


In [163]:
df_reset.to_parquet('../data/raw/domain_data_with_id.parquet', index=False)

In [164]:
sdf = spark.read.parquet('../data/raw/domain_data_with_id.parquet')
sdf.limit(15)

property_url,name,cost_text,rooms,parking,description,prop_type,additional_features
https://www.domai...,1001/39 Queen Str...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],This stunning res...,Apartment / Unit ...,NULL
https://www.domai...,2817s/220 Spencer...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],Stunning Furnishe...,Apartment / Unit ...,[Furnished]
https://www.domai...,3207/80 A'Beckett...,$760/w FURNISHED,"[2 Beds, 1 Bath]",[− Parking],The MY80 developm...,Apartment / Unit ...,"[Gym, Intercom, B..."
https://www.domai...,1413/199 William ...,$650,"[2 Beds, 1 Bath]",[− Parking],Excellent FULLY F...,Apartment / Unit ...,"[Balcony, Outdoor..."
https://www.domai...,2505/371 Little L...,$550,"[1 Bed, 1 Bath]",[− Parking],380 Melbourne ele...,Apartment / Unit ...,NULL
https://www.domai...,208/547 Flinders ...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],***TO BOOK A TIME...,Apartment / Unit ...,NULL
https://www.domai...,1910/22 - 24 Jane...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],This unfurnished ...,Apartment / Unit ...,[Air conditioning...
https://www.domai...,303/18-20 Bank Pl...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],HOW DO I REGISTER...,Apartment / Unit ...,[Built in wardrob...
https://www.domai...,819/555 Flinders ...,$525 per week NOT...,"[1 Bed, 1 Bath]",[1 Parking],Perched high on t...,Apartment / Unit ...,"[Ducted Cooling, ..."
https://www.domai...,2704/288 Spencer ...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],**Booking an insp...,Apartment / Unit ...,"[Gym, Built in wa..."


In [165]:
sdf.dtypes

[('property_url', 'string'),
 ('name', 'string'),
 ('cost_text', 'string'),
 ('rooms', 'array<string>'),
 ('parking', 'array<string>'),
 ('description', 'string'),
 ('prop_type', 'string'),
 ('additional_features', 'array<string>')]

In [166]:
string_columns = ['property_url', 'name', 'cost_text', 'description', 'prop_type']
for column in string_columns:
    sdf = sdf.withColumn(column, lower(col(column)))

sdf

property_url,name,cost_text,rooms,parking,description,prop_type,additional_features
https://www.domai...,1001/39 queen str...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],this stunning res...,apartment / unit ...,NULL
https://www.domai...,2817s/220 spencer...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],stunning furnishe...,apartment / unit ...,[Furnished]
https://www.domai...,3207/80 a'beckett...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],the my80 developm...,apartment / unit ...,"[Gym, Intercom, B..."
https://www.domai...,1413/199 william ...,$650,"[2 Beds, 1 Bath]",[− Parking],excellent fully f...,apartment / unit ...,"[Balcony, Outdoor..."
https://www.domai...,2505/371 little l...,$550,"[1 Bed, 1 Bath]",[− Parking],380 melbourne ele...,apartment / unit ...,NULL
https://www.domai...,208/547 flinders ...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],***to book a time...,apartment / unit ...,NULL
https://www.domai...,1910/22 - 24 jane...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],this unfurnished ...,apartment / unit ...,[Air conditioning...
https://www.domai...,303/18-20 bank pl...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],how do i register...,apartment / unit ...,[Built in wardrob...
https://www.domai...,819/555 flinders ...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],perched high on t...,apartment / unit ...,"[Ducted Cooling, ..."
https://www.domai...,2704/288 spencer ...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],**booking an insp...,apartment / unit ...,"[Gym, Built in wa..."


In [167]:
sdf.write.parquet("../data/raw/domain_data", mode="overwrite")

In [168]:
duplicates = sdf.groupBy("name", "description").agg(count("property_url").alias("count")).filter(col("count") > 1)

# Join with the original DataFrame to filter out duplicates
sdf = sdf.join(duplicates, on=["name", "description"], how="left_anti")

# Show the DataFrame without duplicates
sdf.count()

11528

In [169]:
# Define a UDF to extract number of beds
def extract_beds(rooms):
    for room in rooms:
        if 'Bed' in room:
            # Extract the number before the word 'Bed'
            match = re.search(r'(\d+)\s*Beds?', room)
            if match:
                return int(match.group(1))
    return None  # Return None if no beds information is found

# Define a UDF to extract number of baths
def extract_baths(rooms):
    for room in rooms:
        if 'Bath' in room:
            # Extract the number before the word 'Bath'
            match = re.search(r'(\d+)\s*Baths?', room)
            if match:
                return int(match.group(1))
    return None  # Return None if no baths information is found

# Register UDFs
extract_beds_udf = udf(extract_beds, IntegerType())
extract_baths_udf = udf(extract_baths, IntegerType())

# Apply UDFs to create new columns
sdf = sdf.withColumn('beds', extract_beds_udf(sdf['rooms']))
sdf = sdf.withColumn('baths', extract_baths_udf(sdf['rooms']))

# Show the DataFrame to verify new columns
sdf

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths
1001/39 queen str...,this stunning res...,https://www.domai...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,3,1
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1
1413/199 william ...,excellent fully f...,https://www.domai...,$650,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Balcony, Outdoor...",2,1
2505/371 little l...,380 melbourne ele...,https://www.domai...,$550,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,NULL,1,1
208/547 flinders ...,***to book a time...,https://www.domai...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,2,1
1910/22 - 24 jane...,this unfurnished ...,https://www.domai...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,[Air conditioning...,1,1
303/18-20 bank pl...,how do i register...,https://www.domai...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[Built in wardrob...,1,1
819/555 flinders ...,perched high on t...,https://www.domai...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,"[Ducted Cooling, ...",1,1
2704/288 spencer ...,**booking an insp...,https://www.domai...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Built in wa...",1,1


In [170]:
# Define a UDF to extract number of beds
def extract_parkings(parkings):
    for parking in parkings:
        if 'Parking' in parking:
            # Extract the number before the word 'Bed'
            match = re.search(r'(\d+)\s*Parking?', parking)
            if match:
                return int(match.group(1))
    return None

# Register UDFs
extract_parkings_udf = udf(extract_parkings, IntegerType())

# Apply UDFs to create new columns
sdf = sdf.withColumn('parkings', extract_parkings_udf(sdf['parking']))

# Show the DataFrame to verify new columns
sdf

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths,parkings
1001/39 queen str...,this stunning res...,https://www.domai...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,3,1,NULL
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2,1
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL
1413/199 william ...,excellent fully f...,https://www.domai...,$650,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Balcony, Outdoor...",2,1,NULL
2505/371 little l...,380 melbourne ele...,https://www.domai...,$550,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,NULL,1,1,NULL
208/547 flinders ...,***to book a time...,https://www.domai...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,2,1,NULL
1910/22 - 24 jane...,this unfurnished ...,https://www.domai...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,[Air conditioning...,1,1,1
303/18-20 bank pl...,how do i register...,https://www.domai...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[Built in wardrob...,1,1,NULL
819/555 flinders ...,perched high on t...,https://www.domai...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,"[Ducted Cooling, ...",1,1,1
2704/288 spencer ...,**booking an insp...,https://www.domai...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Built in wa...",1,1,NULL


In [171]:
# Define a UDF to extract numeric cost from 'cost_text'
def extract_numeric_cost(cost_text):
    if cost_text:
        # Remove commas and search for the first sequence of digits
        match = re.search(r'\d+', cost_text.replace(',', ''))
        if match:
            return int(match.group())  # Convert the found digits to an integer
    return None  # Return None if no digits are found

# Register the UDF with IntegerType as the return type
extract_numeric_cost_udf = udf(extract_numeric_cost, IntegerType())

# Apply UDF to create a new column for cost
sdf = sdf.withColumn('cost', extract_numeric_cost_udf(sdf['cost_text']))

# Show the DataFrame to verify the new 'Cost' column
sdf

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths,parkings,cost
1001/39 queen str...,this stunning res...,https://www.domai...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,3,1,NULL,825
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2,1,800
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760
1413/199 william ...,excellent fully f...,https://www.domai...,$650,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Balcony, Outdoor...",2,1,NULL,650
2505/371 little l...,380 melbourne ele...,https://www.domai...,$550,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,NULL,1,1,NULL,550
208/547 flinders ...,***to book a time...,https://www.domai...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,2,1,NULL,525
1910/22 - 24 jane...,this unfurnished ...,https://www.domai...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,[Air conditioning...,1,1,1,525
303/18-20 bank pl...,how do i register...,https://www.domai...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[Built in wardrob...,1,1,NULL,525
819/555 flinders ...,perched high on t...,https://www.domai...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,"[Ducted Cooling, ...",1,1,1,525
2704/288 spencer ...,**booking an insp...,https://www.domai...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Built in wa...",1,1,NULL,480


In [172]:
# Define the schema for the UDF's return type
address_schema = StructType([
    StructField("unit_floor", StringType(), True),
    StructField("street", StringType(), True),
    StructField("suburb", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True)
])

# Define the UDF to handle complex address formats, including 'suburb state postcode'
def extract_address_components(name):
    # Clean double commas first if any
    name = re.sub(r",,", ",", name)

    # Initialize all components as None
    unit_floor, street, suburb, state, postcode = None, None, None, None, None

    # Split the address based on commas
    parts = [part.strip().lower() for part in name.split(',')]

    # Handle cases based on the number of parts found
    if len(parts) >= 1:
        # Check the last part for 'suburb state postcode'
        suburb_state_postcode = parts[-1]
        state_postcode_match = re.search(r'([a-z]{2,3})\s+(\d{4})$', suburb_state_postcode)
        if state_postcode_match:
            state = state_postcode_match.group(1)
            postcode = state_postcode_match.group(2)
            suburb = suburb_state_postcode[:-(len(state + postcode) + 2)].strip() # just added the plus 2 because melb suburb 

        # Check if there is a unit/floor number or a street
        if len(parts) == 1:
            # Only suburb, state, postcode available
            pass
        elif len(parts) == 2:
            # Could be 'street, suburb state postcode'
            street = parts[0]
        elif len(parts) >= 3:
            # Complex structure with unit/floor number
            unit_floor = parts[0]
            street = ', '.join(parts[1:-1])  # Combine middle parts as street

    return (unit_floor, street, suburb, state, postcode)

# Register the UDF with the schema
extract_address_udf = udf(extract_address_components, address_schema)

# Apply the UDF to create new columns for each address component
sdf = sdf.withColumn("AddressComponents", extract_address_udf(sdf['name']))
sdf = sdf.select(*sdf.columns, "AddressComponents.*")

In [173]:
columns_to_drop = ['AddressComponents']

# Dropping the columns from the DataFrame
sdf = sdf.drop(*columns_to_drop)
sdf

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths,parkings,cost,unit_floor,street,suburb,state,postal_code
1001/39 queen str...,this stunning res...,https://www.domai...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,3,1,NULL,825,NULL,1001/39 queen street,melbourne,vic,3000
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2,1,800,NULL,2817s/220 spencer...,melbourne,vic,3000
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000
1413/199 william ...,excellent fully f...,https://www.domai...,$650,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Balcony, Outdoor...",2,1,NULL,650,NULL,1413/199 william ...,melbourne,vic,3000
2505/371 little l...,380 melbourne ele...,https://www.domai...,$550,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,NULL,1,1,NULL,550,NULL,2505/371 little l...,melbourne,vic,3000
208/547 flinders ...,***to book a time...,https://www.domai...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,NULL,2,1,NULL,525,NULL,208/547 flinders ...,melbourne,vic,3000
1910/22 - 24 jane...,this unfurnished ...,https://www.domai...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,[Air conditioning...,1,1,1,525,NULL,1910/22 - 24 jane...,melbourne,vic,3000
303/18-20 bank pl...,how do i register...,https://www.domai...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[Built in wardrob...,1,1,NULL,525,NULL,303/18-20 bank place,melbourne,vic,3000
819/555 flinders ...,perched high on t...,https://www.domai...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,"[Ducted Cooling, ...",1,1,1,525,NULL,819/555 flinders ...,melbourne,vic,3000
2704/288 spencer ...,**booking an insp...,https://www.domai...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Built in wa...",1,1,NULL,480,NULL,2704/288 spencer ...,melbourne,vic,3000


In [174]:
sdf = sdf.filter(~col("cost_text").rlike("^[^0-9]*$"))

In [175]:
sdf = sdf.filter(~
    (((col("description").contains("car space")) & 
    ((col("beds") == 0) | (col("baths") == 0)) &
    (col("cost") < 100)) | (col("prop_type") == "carspace"))
)
sdf.show(truncate=False)

+---------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------+-----------------+-----------+-----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----+--------+----+----------+-------------------------------+---------+-----+-----------

In [176]:
sdf = sdf.withColumn("additional_features", coalesce(col("additional_features"), array()))

In [177]:
sdf.filter(col("cost") > 3000).show(truncate=False)

+-----------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------+--------------------------------------------------+------------------+-----------+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----+--------+-------+----------+---------------------------+--------------+-----+-----------+
|name                           

In [178]:
# Just a check for the the suburb names

# unique_suburbs = sdf.select("suburb").distinct().collect()

# for row in unique_suburbs:
#     print(row['suburb'])

In [179]:
DIMENSION = 25

In [180]:
# Explode the additional_features array to work with each feature separately
features_df = sdf.withColumn("feature", explode(col("additional_features")))

# Count the occurrences of each feature and get the top 20
unique_features_df = features_df.groupBy("feature").count().orderBy(col("count").desc()).limit(DIMENSION)

# Collect the names of the top 20 features
unique_features = [row['feature'] for row in unique_features_df.collect()]

In [181]:
# Add each unique feature as a column to the original DataFrame
for feature in unique_features:
    sdf = sdf.withColumn(feature, array_contains(col("additional_features"), feature).cast("integer"))

# Optionally, show the DataFrame with new one-hot encoded columns
sdf.show(truncate=False)

# Drop the exploded features DataFrame if no longer needed
features_df.unpersist()

+---------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------+-----------------+-----------+-----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----+--------+----+----------+-------------------------------+---------+-----+-----------

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths,parkings,cost,unit_floor,street,suburb,state,postal_code,feature
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2,1,800,NULL,2817s/220 spencer...,melbourne,vic,3000,Furnished
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Gym
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Intercom
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Built in wardrobes
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Dishwasher
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Split System Heating
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Split System Air ...
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Outdoor Entertain...
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Balcony
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,Swimming Pool


In [182]:
# Function to clean column names
def clean_column_name(column_name):
    # Lowercase all characters
    clean_name = column_name.lower()
    # Replace spaces and special characters with underscores
    clean_name = re.sub(r'\W+', '_', clean_name)
    # Remove leading/trailing underscores that might appear after replacements
    clean_name = re.sub(r'^_|_$', '', clean_name)
    return clean_name

# Get current columns in the DataFrame
current_columns = sdf.columns

# Generate new cleaned column names
new_columns = [clean_column_name(column) for column in current_columns]

# Rename columns in the DataFrame
sdf = sdf.toDF(*new_columns)

# Show the updated DataFrame schema to confirm changes
sdf

name,description,property_url,cost_text,rooms,parking,prop_type,additional_features,beds,baths,parkings,cost,unit_floor,street,suburb,state,postal_code,built_in_wardrobes,dishwasher,air_conditioning,floorboards,secure_parking,intercom,heating,ensuite,balcony_deck,gym,furnished,close_to_shops,close_to_transport,fully_fenced,close_to_schools,ducted_heating,split_system_heating,swimming_pool,remote_garage,balcony,study,garden_courtyard,pets_allowed,internal_laundry,alarm_system
1001/39 queen str...,this stunning res...,https://www.domai...,$825 per week,"[3 Beds, 1 Bath]",[− Parking],apartment / unit ...,[],3,1,NULL,825,NULL,1001/39 queen street,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2817s/220 spencer...,stunning furnishe...,https://www.domai...,$800 per week,"[2 Beds, 2 Baths]",[1 Parking],apartment / unit ...,[Furnished],2,2,1,800,NULL,2817s/220 spencer...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3207/80 a'beckett...,the my80 developm...,https://www.domai...,$760/w furnished,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Intercom, B...",2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0
1413/199 william ...,excellent fully f...,https://www.domai...,$650,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,"[Balcony, Outdoor...",2,1,NULL,650,NULL,1413/199 william ...,melbourne,vic,3000,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2505/371 little l...,380 melbourne ele...,https://www.domai...,$550,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[],1,1,NULL,550,NULL,2505/371 little l...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
208/547 flinders ...,***to book a time...,https://www.domai...,$525 weekly,"[2 Beds, 1 Bath]",[− Parking],apartment / unit ...,[],2,1,NULL,525,NULL,208/547 flinders ...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1910/22 - 24 jane...,this unfurnished ...,https://www.domai...,$525.00 pw,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,[Air conditioning...,1,1,1,525,NULL,1910/22 - 24 jane...,melbourne,vic,3000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
303/18-20 bank pl...,how do i register...,https://www.domai...,$525 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,[Built in wardrob...,1,1,NULL,525,NULL,303/18-20 bank place,melbourne,vic,3000,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
819/555 flinders ...,perched high on t...,https://www.domai...,$525 per week not...,"[1 Bed, 1 Bath]",[1 Parking],apartment / unit ...,"[Ducted Cooling, ...",1,1,1,525,NULL,819/555 flinders ...,melbourne,vic,3000,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2704/288 spencer ...,**booking an insp...,https://www.domai...,$480 per week,"[1 Bed, 1 Bath]",[− Parking],apartment / unit ...,"[Gym, Built in wa...",1,1,NULL,480,NULL,2704/288 spencer ...,melbourne,vic,3000,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [183]:
sdf.select(col("prop_type")).distinct()

prop_type
new house & land
villa
new apartments / ...
apartment / unit ...
block of units
duplex
townhouse
semi-detached
studio
terrace


In [184]:
# Create an instance of StringIndexer
indexer = StringIndexer(inputCol="prop_type", outputCol="prop_type_index")

# Fit the indexer to the DataFrame and transform it
sdf = indexer.fit(sdf).transform(sdf)

# Show the DataFrame with the new numeric 'prop_type_index' column
sdf.select("prop_type", "prop_type_index").show(truncate=False)

+-----------------------+---------------+
|prop_type              |prop_type_index|
+-----------------------+---------------+
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|studio                 |3.0            |
|studio                 |3.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
|apartment / unit / flat|0.0            |
+-----------------------+---------

In [185]:
columns_to_drop = ['cost_text', 'rooms', 'parking', 'description', 'prop_type', 'additional_features']

# Dropping the columns from the DataFrame
sdf = sdf.drop(*columns_to_drop)

# Show the updated DataFrame to confirm that columns are dropped
sdf

name,property_url,beds,baths,parkings,cost,unit_floor,street,suburb,state,postal_code,built_in_wardrobes,dishwasher,air_conditioning,floorboards,secure_parking,intercom,heating,ensuite,balcony_deck,gym,furnished,close_to_shops,close_to_transport,fully_fenced,close_to_schools,ducted_heating,split_system_heating,swimming_pool,remote_garage,balcony,study,garden_courtyard,pets_allowed,internal_laundry,alarm_system,prop_type_index
1001/39 queen str...,https://www.domai...,3,1,NULL,825,NULL,1001/39 queen street,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2817s/220 spencer...,https://www.domai...,2,2,1,800,NULL,2817s/220 spencer...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3207/80 a'beckett...,https://www.domai...,2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0.0
1413/199 william ...,https://www.domai...,2,1,NULL,650,NULL,1413/199 william ...,melbourne,vic,3000,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0.0
2505/371 little l...,https://www.domai...,1,1,NULL,550,NULL,2505/371 little l...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
208/547 flinders ...,https://www.domai...,2,1,NULL,525,NULL,208/547 flinders ...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1910/22 - 24 jane...,https://www.domai...,1,1,1,525,NULL,1910/22 - 24 jane...,melbourne,vic,3000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
303/18-20 bank pl...,https://www.domai...,1,1,NULL,525,NULL,303/18-20 bank place,melbourne,vic,3000,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0
819/555 flinders ...,https://www.domai...,1,1,1,525,NULL,819/555 flinders ...,melbourne,vic,3000,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.0
2704/288 spencer ...,https://www.domai...,1,1,NULL,480,NULL,2704/288 spencer ...,melbourne,vic,3000,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0.0


In [186]:
# Write to Parquet
sdf.write.parquet("../data/curated/domain_data.parquet", mode="overwrite")

In [187]:
properties_df = spark.read.csv(
    "../data/raw/properties_stats.csv",
    header=True,  # Assumes first row is header
    inferSchema=True  # Infers the input schema automatically from data
)
properties_df.show(truncate=False)

+---------+--------------------------------------+------------------------------------------------------------------------+-----------+-----------+-------------+-----+----------------------------------------------+-------------------------------------------------+-----------------+------------------+---------------------+--------------------+
|Postcodes|Address                               |URLS                                                                    |Latitude   |Longitude  |Location Type|Count|Location Name                                 |Location Address                                 |Location Latitude|Location Longitude|Travel Time (minutes)|Travel Distance (km)|
+---------+--------------------------------------+------------------------------------------------------------------------+-----------+-----------+-------------+-----+----------------------------------------------+-------------------------------------------------+-----------------+------------------+---------

In [188]:
# remove the properties amenities with errors
properties_df = properties_df.filter(
    (properties_df['Travel Time (minutes)'] != 'Error') & 
    (properties_df['Travel Distance (km)'] != 'Error')
)

properties_df.show(truncate=False)

+---------+--------------------------------------+------------------------------------------------------------------------+-----------+-----------+-------------+-----+----------------------------------------------+---------------------------------------------+-----------------+------------------+---------------------+--------------------+
|Postcodes|Address                               |URLS                                                                    |Latitude   |Longitude  |Location Type|Count|Location Name                                 |Location Address                             |Location Latitude|Location Longitude|Travel Time (minutes)|Travel Distance (km)|
+---------+--------------------------------------+------------------------------------------------------------------------+-----------+-----------+-------------+-----+----------------------------------------------+---------------------------------------------+-----------------+------------------+---------------------

In [189]:
properties_df.select('Address').distinct().count()

1005

In [190]:
properties_df.select("Location Type").distinct()

Location Type
parks
schools
supermarkets
hospitals
shopping_districts
CBD
train_stations


In [191]:
# Grouping by 'URLS' and 'Location Type'
grouped_df = properties_df.groupBy("URLS", "Location Type").agg(
    F.first('Count').alias('Count'),
    F.mean("Travel Time (minutes)").alias("Average Travel Time"),
    F.mean("Travel Distance (km)").alias("Average Travel Distance")
).orderBy("URLS")

grouped_df = grouped_df.withColumn(
    "Time_Distance_Product",
    F.col("Average Travel Time") * F.col("Average Travel Distance")
)

grouped_df.show(truncate=False)

+-------------------------------------------------------------------------------+------------------+-----+-------------------+-----------------------+---------------------+
|URLS                                                                           |Location Type     |Count|Average Travel Time|Average Travel Distance|Time_Distance_Product|
+-------------------------------------------------------------------------------+------------------+-----+-------------------+-----------------------+---------------------+
|https://www.domain.com.au/1-110-kavanagh-street-southbank-vic-3006-17203996    |shopping_districts|2    |2.25               |0.82                   |1.845                |
|https://www.domain.com.au/1-110-kavanagh-street-southbank-vic-3006-17203996    |train_stations    |1    |4.23               |1.57                   |6.641100000000001    |
|https://www.domain.com.au/1-110-kavanagh-street-southbank-vic-3006-17203996    |schools           |2    |2.465              |1.0      

In [192]:
pivot_df = grouped_df.groupBy("URLS").pivot("Location Type", [
    "parks", "schools", "supermarkets", "hospitals", "shopping_districts", "CBD", "train_stations"
]).agg(
    F.first("Count").alias("count"),
    F.first("Average Travel Time").alias("Average Travel Time (Mins)"),
    F.first("Average Travel Distance").alias("Average Travel Distance (Kms)"),
    F.first("Time_Distance_Product").alias("Time_Distance_Product")
)

count_columns = [col for col in pivot_df.columns if col.endswith('_count')]

# Fill null values in the count columns with 0 (since none was found)
pivot_df = pivot_df.fillna(0, subset=count_columns)

# Identify the non-count columns
non_count_columns = [col for col in pivot_df.columns if not col.endswith('_count')]

# Iterate over the non-count columns to replace nulls with the max value
for col_name in non_count_columns:
    # Calculate the maximum value for the column
    max_val = pivot_df.agg(F.max(F.col(col_name)).alias('max_val')).collect()[0]['max_val']
    
    # Replace nulls with the max value in the current column
    if max_val is not None:  # Only proceed if max value is not None
        pivot_df = pivot_df.withColumn(col_name, F.when(F.col(col_name).isNull(), max_val).otherwise(F.col(col_name)))

pivot_df

# Replace the other ones with the maximum val (since if null means none of them was found so = to the maximum val + a certain amount :/) (Used ChatGpt)

URLS,parks_count,parks_Average Travel Time (Mins),parks_Average Travel Distance (Kms),parks_Time_Distance_Product,schools_count,schools_Average Travel Time (Mins),schools_Average Travel Distance (Kms),schools_Time_Distance_Product,supermarkets_count,supermarkets_Average Travel Time (Mins),supermarkets_Average Travel Distance (Kms),supermarkets_Time_Distance_Product,hospitals_count,hospitals_Average Travel Time (Mins),hospitals_Average Travel Distance (Kms),hospitals_Time_Distance_Product,shopping_districts_count,shopping_districts_Average Travel Time (Mins),shopping_districts_Average Travel Distance (Kms),shopping_districts_Time_Distance_Product,CBD_count,CBD_Average Travel Time (Mins),CBD_Average Travel Distance (Kms),CBD_Time_Distance_Product,train_stations_count,train_stations_Average Travel Time (Mins),train_stations_Average Travel Distance (Kms),train_stations_Time_Distance_Product
https://www.domai...,34,1.1300000000000001,0.48,0.5424,2,2.465,1.0,2.465,15,2.816666666666667,1.036,2.918066666666667,0,5.69,1.8766666666666667,10.5265,2,2.25,0.82,1.845,1,5.47,2.13,11.6511,1,4.23,1.57,6.641100000000001
https://www.domai...,0,12.12,9.48,114.8976,1,1.93,0.68,1.3124,21,2.4952380952380957,1.014761904761905,2.5320725623582776,5,3.3825000000000003,1.315,4.4479875,9,1.7550000000000001,0.6575,1.1539125000000001,1,2.16,0.73,1.5768,3,2.07,0.85,1.7594999999999998
https://www.domai...,28,4.23,1.49,6.302700000000001,7,9.585,9.569999999999999,91.72845,9,2.422222222222222,0.9688888888888888,2.3468641975308637,0,5.69,1.8766666666666667,10.5265,4,3.3200000000000003,1.205,4.0006,1,5.21,2.4,12.504,2,1.28,0.52,0.6656000000000001
https://www.domai...,30,2.425,0.725,1.7581249999999997,1,3.45,1.15,3.9675,15,4.212857142857143,1.365,5.75055,0,5.69,1.8766666666666667,10.5265,7,6.196666666666666,2.263333333333333,14.02512222222222,1,6.53,2.32,15.1496,2,5.609999999999999,2.01,11.276099999999998
https://www.domai...,19,3.5500000000000003,1.4849999999999999,5.27175,2,1.97,0.7150000000000001,1.4085500000000002,18,2.865294117647059,1.1982352941176468,3.4332965397923876,0,5.69,1.8766666666666667,10.5265,10,3.26,1.305,4.2543,1,2.42,1.04,2.5168,4,4.59,1.92,8.8128
https://www.domai...,30,2.425,0.725,1.7581249999999997,1,3.45,1.15,3.9675,15,4.212857142857143,1.365,5.75055,0,5.69,1.8766666666666667,10.5265,7,6.196666666666666,2.263333333333333,14.02512222222222,1,6.53,2.32,15.1496,2,5.609999999999999,2.01,11.276099999999998
https://www.domai...,0,12.12,9.48,114.8976,5,3.414,1.5220000000000002,5.196108000000001,1,2.47,1.06,2.6182000000000003,1,2.47,1.11,2.7417000000000002,0,8.08,4.273333333333333,34.528533333333336,1,9.1,3.97,36.127,0,11.6,8.68,100.68799999999999
https://www.domai...,32,1.44,0.54,0.7776000000000001,1,2.18,0.86,1.8748,13,3.193076923076923,1.18,3.767830769230769,0,5.69,1.8766666666666667,10.5265,5,5.515000000000001,2.025,11.167875,1,5.68,2.1,11.927999999999999,1,3.56,1.27,4.5212
https://www.domai...,0,12.12,9.48,114.8976,3,3.5866666666666673,1.1566666666666665,4.148577777777778,20,2.406,0.9040000000000001,2.1750240000000005,6,2.726,1.0539999999999998,2.8732039999999994,10,2.6700000000000004,1.126,3.0064200000000003,1,2.67,1.19,3.1773,0,11.6,8.68,100.68799999999999
https://www.domai...,30,2.365,0.855,2.022075,3,2.4233333333333333,1.1533333333333333,2.794911111111111,10,1.848,0.709,1.310232,0,5.69,1.8766666666666667,10.5265,3,2.335,1.09,2.54515,1,5.06,2.47,12.4982,0,11.6,8.68,100.68799999999999


In [193]:
# Join grouped_df with sdf on 'URLS'
joined_df = sdf.join(pivot_df, sdf.property_url == pivot_df.URLS, how="left").drop("URLS")
joined_df

# Maybe remove the instances where the CBD count wasnt calculated?? a lot now but not too many now 



name,property_url,beds,baths,parkings,cost,unit_floor,street,suburb,state,postal_code,built_in_wardrobes,dishwasher,air_conditioning,floorboards,secure_parking,intercom,heating,ensuite,balcony_deck,gym,furnished,close_to_shops,close_to_transport,fully_fenced,close_to_schools,ducted_heating,split_system_heating,swimming_pool,remote_garage,balcony,study,garden_courtyard,pets_allowed,internal_laundry,alarm_system,prop_type_index,parks_count,parks_Average Travel Time (Mins),parks_Average Travel Distance (Kms),parks_Time_Distance_Product,schools_count,schools_Average Travel Time (Mins),schools_Average Travel Distance (Kms),schools_Time_Distance_Product,supermarkets_count,supermarkets_Average Travel Time (Mins),supermarkets_Average Travel Distance (Kms),supermarkets_Time_Distance_Product,hospitals_count,hospitals_Average Travel Time (Mins),hospitals_Average Travel Distance (Kms),hospitals_Time_Distance_Product,shopping_districts_count,shopping_districts_Average Travel Time (Mins),shopping_districts_Average Travel Distance (Kms),shopping_districts_Time_Distance_Product,CBD_count,CBD_Average Travel Time (Mins),CBD_Average Travel Distance (Kms),CBD_Time_Distance_Product,train_stations_count,train_stations_Average Travel Time (Mins),train_stations_Average Travel Distance (Kms),train_stations_Time_Distance_Product
1001/39 queen str...,https://www.domai...,3,1,NULL,825,NULL,1001/39 queen street,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2817s/220 spencer...,https://www.domai...,2,2,1,800,NULL,2817s/220 spencer...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,17,3.296666666666667,1.4133333333333333,4.659288888888889,3,2.453333333333333,1.0333333333333334,2.535111111111111,17,2.4400000000000004,1.088125,2.6550250000000006,0,5.69,1.8766666666666667,10.5265,9,2.8579999999999997,1.198,3.4238839999999993,1,2.99,1.28,3.8272000000000004,3,3.23,1.5,4.845
3207/80 a'beckett...,https://www.domai...,2,1,NULL,760,NULL,3207/80 a'beckett...,melbourne,vic,3000,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0.0,0,12.12,9.48,114.8976,6,10.231666666666667,9.791666666666666,100.18506944444445,23,1.8191304347826083,0.7408695652173912,1.3477383742911149,0,5.69,1.8766666666666667,10.5265,10,2.2620000000000005,0.8379999999999999,1.895556,1,1.75,0.61,1.0675,0,11.6,8.68,100.68799999999999
1413/199 william ...,https://www.domai...,2,1,NULL,650,NULL,1413/199 william ...,melbourne,vic,3000,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2505/371 little l...,https://www.domai...,1,1,NULL,550,NULL,2505/371 little l...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,12.12,9.48,114.8976,6,10.648333333333335,9.886666666666667,105.27652222222224,28,2.386428571428571,0.9214285714285716,2.198923469387755,0,5.69,1.8766666666666667,10.5265,11,2.375,0.8566666666666666,2.034583333333333,1,1.57,0.48,0.7536,4,5.265000000000001,2.2199999999999998,11.6883
208/547 flinders ...,https://www.domai...,2,1,NULL,525,NULL,208/547 flinders ...,melbourne,vic,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1910/22 - 24 jane...,https://www.domai...,1,1,1,525,NULL,1910/22 - 24 jane...,melbourne,vic,3000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
303/18-20 bank pl...,https://www.domai...,1,1,NULL,525,NULL,303/18-20 bank place,melbourne,vic,3000,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NUL

In [194]:
# Write to Parquet
joined_df.write.parquet("../data/curated/prop_data.parquet", mode="overwrite")

## NULL ANALYSIS

In [195]:
sdf = spark.read.parquet("../data/curated/prop_data.parquet")

In [196]:
# Calculating the count of nulls in each column
null_counts = joined_df.select([count(when(isnull(c), c)).alias(c) for c in joined_df.columns])

# Display the count of nulls per column
null_counts.show(truncate=False)

+----+------------+----+-----+--------+----+----------+------+------+-----+-----------+------------------+----------+----------------+-----------+--------------+--------+-------+-------+------------+---+---------+--------------+------------------+------------+----------------+--------------+--------------------+-------------+-------------+-------+-----+----------------+------------+----------------+------------+---------------+-----------+--------------------------------+-----------------------------------+---------------------------+-------------+----------------------------------+-------------------------------------+-----------------------------+------------------+---------------------------------------+------------------------------------------+----------------------------------+---------------+------------------------------------+---------------------------------------+-------------------------------+------------------------+---------------------------------------------+----------

In [197]:
# location_columns = ["parks", "schools", "supermarkets", "hospitals", "shopping_districts", "CBD", "train_stations"]

# # Calculate min and max for each column and store in a dictionary
# ranges = {
#     c: (sdf.agg({c: "min"}).first()[0], sdf.agg({c: "max"}).first()[0])
#     for c in location_columns
# }

# # Apply a random value within the range for each column
# for column, (min_val, max_val) in ranges.items():
#     # Generate a random value between min and max for each row
#     sdf = sdf.withColumn(column, when(col(column).isNull(), 
#                                        (lit(min_val) + (lit(max_val) - lit(min_val)) * rand())).otherwise(col(column)))

# # Show the DataFrame to confirm changes
# sdf.show(truncate=False)

In [198]:
sdf = sdf.fillna({'unit_floor': 1, 'beds': 1, 'baths': 1, 'parkings': 1, 'street': 'empty'})


In [199]:
csv_df = spark.read.csv("../data/curated/postcode_data.csv", header=True, inferSchema=True)

csv_df = csv_df.withColumnRenamed("poa_code", "postal_code")

csv_df = csv_df.withColumn(
    "pop_percentage_increase", 
    (F.col("2023_pop") - F.col("2022_pop")) / F.col("2022_pop") * 100
)

# think income is by year so divide it by 52 to get weekly income 
csv_df = csv_df.select("median_inc", "2022_pop", "2023_pop", 'pop_percentage_increase', 'postal_code')

sdf = sdf.join(csv_df, on="postal_code", how="inner")

sdf.show(1)


+-----------+--------------------+--------------------+----+-----+--------+----+----------+--------------------+---------+-----+------------------+----------+----------------+-----------+--------------+--------+-------+-------+------------+---+---------+--------------+------------------+------------+----------------+--------------+--------------------+-------------+-------------+-------+-----+----------------+------------+----------------+------------+---------------+-----------+--------------------------------+-----------------------------------+---------------------------+-------------+----------------------------------+-------------------------------------+-----------------------------+------------------+---------------------------------------+------------------------------------------+----------------------------------+---------------+------------------------------------+---------------------------------------+-------------------------------+------------------------+---------------

In [200]:
### Remove the instances with NA, (a lot are being lost rn but once we have more distance data, should be a lot better)
sdf = sdf.dropna() 

In [201]:
null_counts = sdf.select([count(when(isnull(c), c)).alias(c) for c in sdf.columns])

# Display the count of nulls per column
null_counts.show(truncate=False)

sdf.show(1)

+-----------+----+------------+----+-----+--------+----+----------+------+------+-----+------------------+----------+----------------+-----------+--------------+--------+-------+-------+------------+---+---------+--------------+------------------+------------+----------------+--------------+--------------------+-------------+-------------+-------+-----+----------------+------------+----------------+------------+---------------+-----------+--------------------------------+-----------------------------------+---------------------------+-------------+----------------------------------+-------------------------------------+-----------------------------+------------------+---------------------------------------+------------------------------------------+----------------------------------+---------------+------------------------------------+---------------------------------------+-------------------------------+------------------------+---------------------------------------------+----------

In [202]:
# Used Chatgpt to debug this 

### Livability Metric creation (Higher score the better, then will standardise? so its a score between 1 and 100)
livability_weight = {"hospitals_count" : 1, "shopping_districts_count": 3, "parks_count": 2, "schools_count": 4, "supermarkets_count": 4, "train_stations_count": 4}

def harmonic_sum(n):
    if n is None or n <= 0:
        return 0.0
    return sum(1 / i for i in range(1, n + 1))

# Register the UDF
harmonic_sum_udf = F.udf(harmonic_sum, FloatType())

# Initialize the livability score calculation
livability_score_expr = F.lit(0)  # Start with zero

# Iterate through the weights and create an expression to sum the weighted harmonic sums
for count_col, weight in livability_weight.items():
    if count_col in sdf.columns:
        # Calculate the harmonic sum for the count and multiply by the weight
        livability_score_expr += harmonic_sum_udf(F.col(count_col)) * weight

# Add the livability score as a new column in the DataFrame
sdf = sdf.withColumn("livability_score", livability_score_expr)

# Step 1: Calculate the minimum and maximum livability scores
min_max = sdf.agg(
    F.min("livability_score").alias("min_score"),
    F.max("livability_score").alias("max_score")
).first()

min_score = min_max['min_score']
max_score = min_max['max_score']

# Step 2: Apply min-max normalization to create a new column
if min_score is not None and max_score is not None and min_score != max_score:
    sdf = sdf.withColumn(
        "normalized_livability_score",
        ((F.col("livability_score") - min_score) / (max_score - min_score) * 100).cast("int")
    )
else:
    # If all values are the same, set normalized score to 0 or handle accordingly
    sdf = sdf.withColumn("normalized_livability_score", F.lit(0).cast("int"))

sdf = sdf.drop("livability_score")

# Show the updated DataFrame
sdf.show(truncate=False)


+-----------+---------------------------------------------------+-------------------------------------------------------------------------------------+----+-----+--------+----+----------+-------------------------------+---------+-----+------------------+----------+----------------+-----------+--------------+--------+-------+-------+------------+---+---------+--------------+------------------+------------+----------------+--------------+--------------------+-------------+-------------+-------+-----+----------------+------------+----------------+------------+---------------+-----------+--------------------------------+-----------------------------------+---------------------------+-------------+----------------------------------+-------------------------------------+-----------------------------+------------------+---------------------------------------+------------------------------------------+----------------------------------+---------------+------------------------------------+-----

In [203]:
# just some renaming 

feature_names = [
    "postal_code",
    "name",
    "property_url",
    "beds",
    "baths",
    "parkings",
    "cost",
    "built_in_wardrobes",
    "dishwasher",
    "air_conditioning",
    "floorboards",
    "secure_parking",
    "intercom",
    "heating",
    "ensuite",
    "balcony_deck",
    "gym",
    "furnished",
    "close_to_shops",
    "close_to_transport",
    "fully_fenced",
    "close_to_schools",
    "ducted_heating",
    "split_system_heating",
    "swimming_pool",
    "remote_garage",
    "balcony",
    "study",
    "garden_courtyard",
    "pets_allowed",
    "internal_laundry",
    "alarm_system",
    "prop_type_index",
    "parks_count",
    "parks_Average Travel Time (Mins)",
    "parks_Average Travel Distance (Kms)",
    "parks_Time_Distance_Product",
    "schools_count",
    "schools_Average Travel Time (Mins)",
    "schools_Average Travel Distance (Kms)",
    "schools_Time_Distance_Product",
    "supermarkets_count",
    "supermarkets_Average Travel Time (Mins)",
    "supermarkets_Average Travel Distance (Kms)",
    "supermarkets_Time_Distance_Product",
    "hospitals_count",
    "hospitals_Average Travel Time (Mins)",
    "hospitals_Average Travel Distance (Kms)",
    "hospitals_Time_Distance_Product",
    "shopping_districts_count",
    "shopping_districts_Average Travel Time (Mins)",
    "shopping_districts_Average Travel Distance (Kms)",
    "shopping_districts_Time_Distance_Product",
    "CBD_Average Travel Distance (Kms)",
    "train_stations_count",
    "train_stations_Average Travel Time (Mins)",
    "train_stations_Average Travel Distance (Kms)",
    "train_stations_Time_Distance_Product",
    "median_inc",
    "2022_pop",
    "2023_pop",
    "pop_percentage_increase",
    "normalized_livability_score"
]

sdf = sdf.select(feature_names)



In [204]:
sdf.write.parquet("../data/curated/prop_data_cleaned.parquet", mode="overwrite")

In [205]:
spark.stop()